In [1]:
# Check on gpu
!nvidia-smi

Wed Apr 23 12:57:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100S-PCIE-32GB          On  |   00000000:18:00.0 Off |                    0 |
| N/A   30C    P0             26W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Import necessary libraries
import os
import numpy as np
from glob import glob
from tifffile import imread, imwrite
from tqdm import tqdm

from stardist.models import StarDist2D
from stardist import random_label_cmap
from csbdeep.utils import normalize

2025-04-23 12:57:35.852037: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 12:57:36.331128: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-23 12:57:43.138732: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
# Check for GPU and configure TensorFlow
if gpus:
    print(f"GPU detected: {gpus}") # StarDist automatically uses GPU if it is available
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow set to use GPU with memory growth.")
    except RuntimeError as e:
        print(f"Could not set memory growth: {e}")
else:
    print("No GPU detected. StarDist will use CPU.")


GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow set to use GPU with memory growth.


In [8]:
# Set Stardist model and parameters
model = StarDist2D.from_pretrained('2D_versatile_fluo')
prob_thresh = 0.5
overlap_thresh = 0.4

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [9]:
# Segmentation
# Set the master folder path here
master_folder = '/home/vil945/live_cell_imaging/2025-03-26_live_cell_imaging'

# Recursively find folders with '_CTV' or '_PI' in their names
target_folders = []
for root, dirs, files in os.walk(master_folder):
    for dir_name in dirs:
        if '_CTV' in dir_name or '_PI' in dir_name:
            target_folders.append(os.path.join(root, dir_name))

# Process each matching folder
for folder in tqdm(target_folders, desc="Processing target folders"):
    input_files = glob(os.path.join(folder, '*.tif'))
    if not input_files:
        continue  # Skip empty folders

    # Create output folder inside the current one
    folder_name = os.path.basename(folder)
    output_folder = os.path.join(folder, f'{folder_name}_segmented')
    os.makedirs(output_folder, exist_ok=True)

    for filepath in tqdm(input_files, desc=f"Segmenting in {folder_name}", leave=False):
        # Read multi-frame TIFF
        img_stack = imread(filepath)
        n_frames = img_stack.shape[0]

        label_stack = []

        # Segment each frame
        for i in range(n_frames):
            img = normalize(img_stack[i], 1, 99.8)
            labels = model.predict_instances(img, prob_thresh=prob_thresh, nms_thresh=overlap_thresh)[0]
            label_stack.append(labels)

        # Save labeled multi-frame image
        label_stack = np.stack(label_stack).astype(np.uint16)
        filename = os.path.basename(filepath)
        save_path = os.path.join(output_folder, f'segmented_{filename}')
        imwrite(save_path, label_stack)

print("All segmentations complete.")

Segmenting in 2025-03-26_CTV:   0%|          | 0/4 [00:00<?, ?it/s]2025-04-23 13:02:05.065002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700

Processing target folders: 100%|██████████| 2/2 [04:52<00:00, 146.26s/it] 

All segmentations complete.
